In [5]:
import pkg_resources
for d in pkg_resources.working_set:
     if str(d)[0:3] == "ff3" or str(d)[0:12] == "pycryptodome" or str(d)[0:11] == "unicodedata":
          print(str(d))

StatementMeta(testhmacspark, 67, 2, Finished, Available)

unicodedata2 15.0.0
pycryptodome 3.17
ff3 1.0.1


In [6]:
%%pyspark
#df = spark.read.load.('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt', format='text', header='true')
df = spark.read.option("header",True).csv('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer_1M.txt')

display(df.limit(10))

# display(df.groupBy("MiddleName").count().orderBy("MiddleName"))
# https://files.pythonhosted.org/packages/14/58/77278d7a078241b55b515f6073b90108125fb0d197b384a0f372c5f61c80/pycryptodome-3.17-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# https://www.wheelodex.org/projects/ff3/wheels/ff3-1.0.1-py3-none-any.whl/
# https://files.pythonhosted.org/packages/be/ea/90e14e807da5a39e5b16789acacd48d63ca3e4f23dfa964a840eeadebb13/Unidecode-1.3.6-py3-none-any.whl

StatementMeta(testhmacspark, 67, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1ab1d5ce-be5e-4a22-9fba-75a2c5a6f260)

In [7]:
from ff3 import FF3Cipher
import string
#
KEY = TokenLibrary.getSecret('test-hmacmask-akv', 'fpekey', 'AzureKeyVaultHMAC')
TWEAK = TokenLibrary.getSecret('test-hmacmask-akv', 'fpetweak', 'AzureKeyVaultHMAC')
#
C_NUMERIC_MIN = 6
C_NUMERIC_MAX = 56
C_NUMERIC_RADIX = 10
C_NUMERIC_MIN_PREFIX = "0"
#
C_ALPHANUMERIC_MIN = 6
C_ALPHANUMERIC_MAX = 32
C_ALPHANUMERIC_RADIX = 62
C_ALPHANUMERIC_MIN_PREFIX = "0"
#
C_ALPHA_LOWER_MIN = 5
C_ALPHA_LOWER_MAX = 40
C_ALPHA_LOWER_MIN_PREFIX = "a"
C_ALPHA_LOWER_ALPHABET = "abcdefghijklmnopqrstuvwxyz"
#
C_ALPHA_UPPER_MIN = 5
C_ALPHA_UPPER_MAX = 40
C_ALPHA_UPPER_MIN_PREFIX = "A"
C_ALPHA_UPPER_ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
#
C_ALPHANUMERIC_EXTENDED_MIN = 4
C_ALPHANUMERIC_EXTENDED_MAX = 30
C_ALPHANUMERIC_EXTENDED_MIN_PREFIX = " "
C_ALPHANUMERIC_EXTENDED_ALPHABET = string.digits + string.ascii_lowercase + string.ascii_uppercase + " !ç@#$%^&*()?'\/-.,¡é"
#
C_EMAIL_MIN = 4
C_EMAIL_MAX = 30
C_EMAIL_MIN_PREFIX = " "
C_EMAIL_ALPHABET = string.digits + string.ascii_lowercase + string.ascii_uppercase + "._%+-çé"

StatementMeta(testhmacspark, 67, 4, Finished, Available)

In [8]:
from unidecode import unidecode

def _fpe_ff3_base(col_val, c, c_min, c_max, c_min_prefix):

    if col_val == None:
        return None

    ciphertext =""
    if len(col_val) < c_min:
        # column length is too short to be encrypted. Prefix column with values to reach minimal length
        col_val = str(col_val).rjust(c_min, c_min_prefix)
        ciphertext = c.encrypt(col_val)
    elif len(col_val) > c_max:
        # column length is too long and needs to be chunked. Every chunck is then encrypted and appended
        current_chunck_length = c_max
        while current_chunck_length > 0:
            col_val_array = [col_val[i:i+current_chunck_length] for i in range(0, len(col_val), current_chunck_length)]
            # check if length of last chunck is larger than minimum
            if len(col_val_array[-1]) < c_min:
                #length of last element is shorter than minimum. Decrease current_chunck_length with 1
                current_chunck_length -= 1
            else:
                # length of last chunck is larger than minimum. Ready process
                break
        
        for col_val_item in col_val_array:
            ciphertext += c.encrypt(col_val_item)
    else:
        ciphertext = c.encrypt(col_val)

    return ciphertext

def _fpe_ff3_phone_parsing(col_val, c, c_min, c_max, c_min_prefix):

    if col_val == None:
        return None

    raw_string = str(''.join(filter(str.isdigit, col_val)))
    raw_ciphertext = _fpe_ff3_base(raw_string, c, c_min, c_max, c_min_prefix)

    ciphertext = ""
    numeric_count = 0
    for j, val in enumerate(col_val):
        if val.isnumeric():
            ciphertext += raw_ciphertext[numeric_count]
            numeric_count += 1
        else:
            ciphertext += col_val[j]
    
    return ciphertext

def _fpe_ff3_email_parsing(col_val, c, c_min, c_max, c_min_prefix):

    if col_val == None:
        return None

    at_split = col_val.split("@")
    len_at_split = len(at_split[0])
    dot_split = at_split[1].split(".")
    len_dot_split = len_at_split + len(dot_split[0])
    raw_string = at_split[0] + dot_split[0]
    raw_ciphertext = _fpe_ff3_base(raw_string, c, c_min, c_max, c_min_prefix)
    ciphertext = raw_ciphertext[0 : len_at_split] + "@" + \
                 raw_ciphertext[len_at_split : len_dot_split] + "." + \
                 col_val[len_dot_split + 2 : ]
    
    return ciphertext

def _fpe_ff3_to_ascii_preserve_other_parsing(col_val, \
                                    c_alpha_lower, c_alpha_lower_min, c_alpha_lower_max, c_alpha_lower_min_prefix, \
                                    c_alpha_upper, c_alpha_upper_min, c_alpha_upper_max, c_alpha_upper_min_prefix, \
                                    c_numeric, c_numeric_min, c_numeric_max, c_numeric_min_prefix):

    if col_val == None:
        return None

    col_val = unidecode(col_val) # convert unicode to ascii, č becomes c   
    raw_alpha = str(''.join(filter(str.isalpha, col_val)))
    raw_alpha_lower = str(''.join(filter(str.islower, raw_alpha)))
    raw_alpha_upper = str(''.join(filter(str.isupper, raw_alpha)))
    raw_numeric = str(''.join(filter(str.isdigit, col_val)))
    #
    raw_alpha_lower_ciphertext = _fpe_ff3_base(raw_alpha_lower, c_alpha_lower, c_alpha_lower_min, c_alpha_lower_max, c_alpha_lower_min_prefix)
    raw_alpha_upper_ciphertext = _fpe_ff3_base(raw_alpha_upper, c_alpha_upper, c_alpha_upper_min, c_alpha_upper_max, c_alpha_upper_min_prefix)
    raw_numeric_ciphertext = _fpe_ff3_base(raw_numeric, c_numeric, c_numeric_min, c_numeric_max, c_numeric_min_prefix)

    ciphertext = ""
    alpha_lower_count = 0
    alpha_upper_count = 0
    numeric_count = 0
    for j, val in enumerate(col_val):
        if val.isalpha():
            if val.islower():
                ciphertext += raw_alpha_lower_ciphertext[alpha_lower_count]
                alpha_lower_count += 1
            else:
                ciphertext += raw_alpha_upper_ciphertext[alpha_upper_count]
                alpha_upper_count += 1            
        elif val.isnumeric():
            ciphertext += raw_numeric_ciphertext[numeric_count]
            numeric_count += 1
        else:
            ciphertext += col_val[j]
    
    return ciphertext    

def fpe_ff3_numeric(col_val):
    c = FF3Cipher(KEY, TWEAK, radix=C_NUMERIC_RADIX)
    return _fpe_ff3_base(col_val, c, C_NUMERIC_MIN, C_NUMERIC_MAX, C_NUMERIC_MIN_PREFIX)

def fpe_ff3_alphanumeric(col_val):
    c = FF3Cipher(KEY, TWEAK, radix=C_ALPHANUMERIC_RADIX)
    return _fpe_ff3_base(col_val, c, C_ALPHANUMERIC_MIN, C_ALPHANUMERIC_MAX, C_ALPHANUMERIC_MIN_PREFIX)

def fpe_ff3_alphanumeric_extended(col_val):
    c = FF3Cipher.withCustomAlphabet(KEY, TWEAK, alphabet=C_ALPHANUMERIC_EXTENDED_ALPHABET)
    return _fpe_ff3_base(col_val, c, C_ALPHANUMERIC_EXTENDED_MIN, C_ALPHANUMERIC_EXTENDED_MAX, C_ALPHANUMERIC_EXTENDED_MIN_PREFIX)

def fpe_ff3_phone(col_val):
    c = FF3Cipher(KEY, TWEAK, radix=C_NUMERIC_RADIX)
    return _fpe_ff3_phone_parsing(col_val, c, C_NUMERIC_MIN, C_NUMERIC_MAX, C_NUMERIC_MIN_PREFIX)

def fpe_ff3_email(col_val):
    c = FF3Cipher.withCustomAlphabet(KEY, TWEAK, alphabet=C_EMAIL_ALPHABET)
    return _fpe_ff3_email_parsing(col_val, c, C_EMAIL_MIN, C_EMAIL_MAX, C_EMAIL_MIN_PREFIX)

def fpe_ff3_to_ascii_preserve_other(col_val):
    c_alpha_lower = FF3Cipher.withCustomAlphabet(KEY, TWEAK, alphabet=C_ALPHA_LOWER_ALPHABET)
    c_alpha_upper = FF3Cipher.withCustomAlphabet(KEY, TWEAK, alphabet=C_ALPHA_UPPER_ALPHABET)
    c_numeric = FF3Cipher(KEY, TWEAK, radix=C_NUMERIC_RADIX)
    return _fpe_ff3_to_ascii_preserve_other_parsing(col_val, \
                                                       c_alpha_lower, C_ALPHA_LOWER_MIN, C_ALPHA_LOWER_MAX, C_ALPHA_LOWER_MIN_PREFIX, \
                                                       c_alpha_upper, C_ALPHA_UPPER_MIN, C_ALPHA_UPPER_MAX, C_ALPHA_UPPER_MIN_PREFIX, \
                                                       c_numeric, C_NUMERIC_MIN, C_NUMERIC_MAX, C_NUMERIC_MIN_PREFIX)

StatementMeta(testhmacspark, 67, 5, Finished, Available)

In [10]:
id = "12345232"
id_fpe = fpe_ff3_numeric(id)
name = "Bremer"
name_fpe = fpe_ff3_alphanumeric(name)
company = "Bremer & Sons!, LTD."
company_fpe = fpe_ff3_alphanumeric_extended(company)
phone = "06-23112312"
phone_fpe = fpe_ff3_phone(phone)
email = "bremersons@hotmail.com"
email_fpe = fpe_ff3_email(email)
to_ascii_preserve_other = "Kožušček123a"
to_ascii_preserve_other_fpe = fpe_ff3_to_ascii_preserve_other(to_ascii_preserve_other)
#
print(id, "=>",  id_fpe)
print(name, "=>",  name_fpe)
print(company, "=>",  company_fpe)
print(phone, "=>",  phone_fpe)
print(email, "=>",  email_fpe)
print(to_ascii_preserve_other, "=>",  to_ascii_preserve_other_fpe)

StatementMeta(testhmacspark, 67, 7, Finished, Available)

12345232 => 73766379
Bremer => 6paCYa
Bremer & Sons!, LTD. => OsH0*VlF(dsIGHXkZ4dK
06-23112312 => 48-78322271
bremersons@hotmail.com => l-YLt53VGs@VDIdçQ4.com
Kožušček123a => Sqxbblkd659p


In [11]:
# https://tunetotech.com/Post.aspx?post=9
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from ff3 import FF3Cipher
#
fpe_ff3_numeric_udf = udf(fpe_ff3_numeric, StringType())
fpe_ff3_alphanumeric_udf = udf(fpe_ff3_alphanumeric, StringType())
fpe_ff3_alphanumeric_extended_udf = udf(fpe_ff3_alphanumeric_extended, StringType())
fpe_ff3_email_udf = udf(fpe_ff3_email, StringType())
fpe_ff3_phone_udf = udf(fpe_ff3_phone, StringType())
fpe_ff3_to_ascii_preserve_other_udf = udf(fpe_ff3_to_ascii_preserve_other, StringType())
#
df2=df.withColumn('CustomerID', fpe_ff3_numeric_udf(df.CustomerID)) \
      .withColumn('FirstName', fpe_ff3_alphanumeric_extended_udf(df.FirstName)) \
      .withColumn('LastName', fpe_ff3_alphanumeric_extended_udf(df.LastName)) \
      .withColumn('CompanyName', fpe_ff3_alphanumeric_extended_udf(df.CompanyName)) \
      .withColumn('EmailAddress', fpe_ff3_email_udf(df.EmailAddress)) \
      .withColumn('Phone', fpe_ff3_phone_udf(df.Phone)) \
      .withColumn('MiddleName', fpe_ff3_to_ascii_preserve_other_udf(df.MiddleName))

display(df2.limit(10))

StatementMeta(testhmacspark, 67, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, ecb949dc-a432-4c40-993d-fc32c7593a28)

In [12]:
df2.write.mode("overwrite").option("header","true").csv('abfss://gold@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer_1M_masked.txt')

StatementMeta(testhmacspark, 67, 9, Finished, Available)